In [14]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # specify which GPU(s) to be used
sys.path.append("./input/prostate-cancer-grade-assessment/")
sys.path.append("./input")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import *
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
# device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu") 
# print(device)

## customized package
# from input.inputPipeline_spine_infer import *
from input.inputPipeline_spine_infer_dataspecific import *
from input.inputPipeline_stiching import crossValDataloader, PandaPatchDataset
from model.resnext_ssl_stiching import Model_Infer as Model

In [15]:
data_dir = './input/prostate-cancer-grade-assessment/train_images'
data_dir_sub = './input/panda-36x256x256-tiles-data-spine/train'
# csv_file = './input/panda-36x256x256-tiles-data-spine/4_fold_whole_train.csv'
csv_file = './input/panda-36x256x256-tiles-data-spine/radboud_4_fold_train.csv'
# SAMPLE = './input/prostate-cancer-grade-assessment/sample_submission.csv'

In [28]:
models = []
# weights = [f'./train/weights/Resnext50_36patch_adam_cos_spine_rad/Resnext50_36patch_adam_cos_spine_rad_{i}_best.pth.tar' for i in [3]]
weights = [f'./train/weights/Resnext50_36patch_adam_cos_spine_loc_pre_rad/Resnext50_36patch_adam_cos_spine_loc_pre_rad_{i}_best.pth.tar' for i in [0]]

for path in weights:
    pretrained_dict = torch.load(path)
    model = Model(n = 5)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    model.load_state_dict(pretrained_dict)
    model.eval()
    model.cuda()
    models.append(model)

del pretrained_dict

In [29]:
sz = 256
bs = 1
folds = 4
tsfm = None
dataset = PandaPatchDatasetInfer(csv_file, data_dir, sz, transform=tsfm, N = 36, rand = False)
# dataset = PandaPatchDataset(csv_file, data_dir_sub, sz, transform=tsfm, N = 36, rand = False)
crossValData = crossValDataloader(csv_file, dataset, bs)

In [30]:
names, val_preds = [], []
# for fold in range(folds):
for fold in [0]:
    _, valloader = crossValData(fold)
#     model = models[fold]
    model = models[fold]
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(valloader, desc='valIter'), start=0):
#             if i > 7:
#                 break
            # get the inputs; data is a list of [inputs, labels]
            img, name, provider = data['img'], data['name'], data['datacenter']
            outputs = model(img.cuda())
            outputs = outputs['out']
            val_preds.append(outputs.sigmoid().sum(1).round().cpu())
            names.append(name)

In [31]:
names = np.concatenate(names)
predictions = list(torch.cat(val_preds, 0).numpy().astype(int))

In [32]:
df = pd.read_csv(csv_file)
grades = df.loc[df['split'] == 0, 'isup_grade'].tolist()
# grades[:len(predictions)]

In [33]:
kappa = cohen_kappa_score(grades[:len(predictions)], predictions, weights='quadratic')
print("The average kappa score is {}.".format(kappa))

The average kappa score is 0.866652640496836.


In [ ]:
df_cm = pd.DataFrame(ncm, range(6), range(6))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.show()